In [5]:
import os
import numpy as np 
import geopandas as gpd
import folium
import matplotlib.colors as colors
from branca.colormap import LinearColormap
from IPython.display import display

# Section 0: Configuration & Setup

GEOJSON_PATH = 'outputs/final_results/rooftop_solar_potential_optimized.geojson'
OUTPUT_MAP_DIR = 'outputs/Fig'
os.makedirs(OUTPUT_MAP_DIR, exist_ok=True)

# Section 1: Data Loading

if not os.path.exists(GEOJSON_PATH):
    print(f"Error: File not found: {GEOJSON_PATH}")
    exit()

print(f"Loading data: {GEOJSON_PATH}")
gdf = gpd.read_file(GEOJSON_PATH)

# Data Cleaning
for col in ['potential_kwh', 'payback_years', 'yearly_savings', 'area_m2']:
    if col in gdf.columns:
        gdf[col] = gdf[col].fillna(0).astype(float)

# Data Reclassification (Zoning Correction for Interactive Map)
# Step 1: Initialize clean zoning column
gdf['zoning_type_clean'] = gdf['zoning_type']

# Step 2: Define area threshold 
AREA_THRESHOLD_FOR_INSTITUTIONAL = 3000 

# Step 3: Identify outliers (Large buildings incorrectly marked as Residential)
is_residential = gdf['zoning_type'].astype(str).str.contains('Resid', case=False, na=False)
is_oversized = gdf['area_m2'] > AREA_THRESHOLD_FOR_INSTITUTIONAL

outlier_mask = is_residential & is_oversized

# Step 4: Reclassify outliers
if outlier_mask.any():
    gdf.loc[outlier_mask, 'zoning_type_clean'] = 'Institutional/Hospital'
    print(f"Successfully reclassified {outlier_mask.sum()} large institutional buildings (outliers).")
else:
    print("No residential outliers found requiring reclassification.")
    
# Coordinate System Preparation
# Folium requires WGS84 (EPSG:4326)
gdf_ll = gdf.to_crs(epsg=4326)

print(f"Data loaded and processed successfully. Total roofs: {len(gdf)}.")

# Section 2: Interactive Web Map Generation

def plot_interactive_web_map(gdf):
    print(f"   Generating: Interactive Web Map ...")
    save_path = os.path.join(OUTPUT_MAP_DIR, 'Fig_Interactive_Website.html')

    center_lat = gdf.geometry.centroid.y.mean()
    center_lon = gdf.geometry.centroid.x.mean()
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles='CartoDB positron')
    
    min_val = gdf['potential_kwh'].min()
    max_val = gdf['potential_kwh'].max()
    if max_val == min_val: max_val = min_val + 1.0
    
    colormap = LinearColormap(
        colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'], 
        vmin=min_val, vmax=max_val
    )
    colormap.caption = 'Annual Solar Potential (kWh/yr)'
    colormap.add_to(m)

    def style_fn(feature):
        try:
            val = feature['properties']['potential_kwh']
            if val is None: val = 0
            return {'fillColor': colormap(val), 'color': 'black', 'weight': 0.5, 'fillOpacity': 0.8}
        except:
            return {'fillColor': 'gray', 'color': 'black', 'weight': 0.5}

    folium.GeoJson(
        gdf,
        name='Solar Potential',
        style_function=style_fn,
        highlight_function=lambda x: {'weight': 3, 'color': '#00BFFF'},
        tooltip=folium.GeoJsonTooltip(
            fields=['zoning_type', 'area_m2', 'potential_kwh', 'yearly_savings', 'payback_years'],
            aliases=['Zone:', 'Area (m²):', 'Potential (kWh):', 'Savings ($):', 'Payback (Yr):'],
            localize=True, sticky=True
        )
    ).add_to(m)
    
    m.save(save_path)
    print(f"      Saved: {save_path}")

# Execute Plotting for Interactive Map
plot_interactive_web_map(gdf_ll)

print("\nProcess Complete! Please check the outputs/Fig folder for the result.")

Loading data: outputs/final_results/rooftop_solar_potential_optimized.geojson
Successfully reclassified 7 large institutional buildings (outliers).
Data loaded and processed successfully. Total roofs: 4608.
   Generating: Interactive Web Map ...


C:\Users\Asus\AppData\Local\Temp\ipykernel_23460\740864558.py:61: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = gdf.geometry.centroid.y.mean()
C:\Users\Asus\AppData\Local\Temp\ipykernel_23460\740864558.py:62: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = gdf.geometry.centroid.x.mean()


      Saved: outputs/Fig\Fig_Interactive_Website.html

Process Complete! Please check the outputs/Fig folder for the result.
